In [1]:
import pandas as pd
import numpy as np
import spacy
pd.set_option('display.max_colwidth', None)

In [14]:
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 13.9 MB 2.4 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     |████████████████████████████████| 777.4 MB 15 kB/s s eta 0:00:013     |████████████████████▍           | 494.5 MB 16.1 MB/s eta 0:00:18     |███████████████████████████     | 655.4 MB 20.9 MB/s eta 0:00:06
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dragonzord/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
patient_notes = pd.read_csv("patient_notes.csv")
patient_notes.head(2)

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std"
1,1,0,"17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. ."


In [5]:
features = pd.read_csv("features.csv")
features.head(2)

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myocardial-infarction
1,1,0,Family-history-of-thyroid-disorder


In [6]:
second_note = patient_notes.iloc[1, 2]
second_feature = features.iloc[1, 2]

In [7]:
print(second_note)
print(second_feature)

17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. .
Family-history-of-thyroid-disorder


In [8]:
case_num_zero = patient_notes["pn_history"][patient_notes["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())

In [9]:
import acquire
clean_note = acquire.basic_clean(whole_case_num_zero)
clean_note = acquire.remove_stopwords(clean_note, exclude_words=["no"])

In [10]:
len(clean_note)

962584

In [16]:
nlp = spacy.load("en_core_web_lg")

In [19]:
doc = nlp(clean_note)

In [20]:
len(doc)

149352

In [ ]:
for token in doc[:1000]:
    print(token.text, token.ent_iob_, token.ent_type_)

In [23]:
for ent in doc[:10000].ents:
    print(f"text: {ent.text}, start_char: {ent.start_char}, end_char: {ent.end_char}, label_: {ent.label_}, start: {ent.start}, ent_id: {ent.ent_id_}, label: {ent.label}, vector_norm: {ent.vector_norm}")

text: 17 year old, start_char: 0, end_char: 11, label_: DATE, start: 0, ent_id: , label: 391, vector_norm: 4.27012300491333
text: 2 3 months ago, start_char: 157, end_char: 171, label_: DATE, start: 22, ent_id: , label: 391, vector_norm: 4.629660606384277
text: 2 days, start_char: 192, end_char: 198, label_: DATE, start: 28, ent_id: , label: 391, vector_norm: 4.556836128234863
text: 3 4 min, start_char: 207, end_char: 214, label_: TIME, start: 31, ent_id: , label: 392, vector_norm: 4.663009166717529
text: mi, start_char: 458, end_char: 460, label_: PERSON, start: 72, ent_id: , label: 380, vector_norm: 7.944142818450928
text: 6 months ago, start_char: 514, end_char: 526, label_: DATE, start: 82, ent_id: , label: 391, vector_norm: 4.978621959686279
text: 17, start_char: 571, end_char: 573, label_: CARDINAL, start: 93, ent_id: , label: 397, vector_norm: 5.423252582550049
text: 3 4 min, start_char: 623, end_char: 630, label_: TIME, start: 102, ent_id: , label: 392, vector_norm: 4.663009166